## [Camera Calibration and 3D Reconstruction](https://docs.opencv.org/3.4/d9/db7/tutorial_py_table_of_contents_calib3d.html)
이 섹션에서는 카메라 보정, 스테레오 이미징 등을 배우게됩니다.

### [Camera Calibration](https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html)
Let's find how good is our camera. Is there any distortion in images taken with it? If so how to correct it?

#### Goal
이 섹션에서는

* 카메라로 인한 왜곡 유형
* 카메라의 본질적 및 외적 속성을 찾는 방법
* 이러한 속성을 기반으로 이미지를 왜곡되지 않게 방법

#### Basics
일부 핀홀 카메라는 이미지에 큰 왜곡을 일으 킵니다. 두 가지 종류의 왜곡은 방사형 왜곡과 접선 왜곡입니다.

방사형 왜곡은 직선이 곡선으로 나타나는 원인이됩니다. 방사형 왜곡은 이미지 중심에서 먼 지점이 클수록 커집니다. 예를 들어 체스 보드의 두 가장자리에 빨간색 선이 표시되는 이미지가 아래에 나와 있습니다. 그러나 체스 판의 경계선이 직선이 아니고 붉은 선과 일치하지 않는 것을 볼 수 있습니다. 예상되는 모든 직선이 돌출되어 있습니다. 자세한 내용은 [왜곡(광학)](https://en.wikipedia.org/wiki/Distortion_%28optics%29) 을 방문 하십시오.

![](calib_radial.jpg) 

Radial distortion can be represented as follows:

\\(x_{distorted} = x( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\ y_{distorted} = y( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)\\)

마찬가지로, 접선 왜곡은 이미지 촬영 렌즈가 이미징 평면과 완벽하게 평행하게 정렬되어 있지 않기 때문에 발생합니다. 따라서 이미지의 일부 영역이 예상보다 가깝게 보일 수 있습니다. 접선 왜곡의 크기는 다음과 같이 나타낼 수 있습니다.

\\(x_{distorted} = x + [ 2p_1xy + p_2(r^2+2x^2)] \\ y_{distorted} = y + [ p_1(r^2+ 2y^2)+ 2p_2xy]\\)

In short, we need to find five parameters, known as distortion coefficients given by:

\\(Distortion \; coefficients=(k_1 \hspace{10pt} k_2 \hspace{10pt} p_1 \hspace{10pt} p_2 \hspace{10pt} k_3)\\)


이 외에도 카메라의 내장 및 외장 매개 변수와 같은 다른 정보가 필요합니다. 내장 매개 변수는 카메라에만 적용됩니다. 초점 거리 \\(((f_x, f_y) \\)및 광학 중심 \\((c_x, c_y)\\)과 같은 정보가 포함됩니다. 초점 거리와 광학 중심을 사용하여 특정 카메라의 렌즈로 인한 왜곡을 제거하는 데 사용할 수있는 카메라 매트릭스를 만들 수 있습니다. 카메라 매트릭스는 특정 카메라에만 적용되므로 한 번 계산하면 동일한 카메라로 촬영 한 다른 이미지에 다시 사용할 수 있습니다. 그것은 3x3 매트릭스로 표현됩니다 :

\\(camera \; matrix = \left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]\\)

외재 매개 변수는 3D 점의 좌표를 좌표계로 변환하는 회전 및 변환 벡터에 해당합니다.

스테레오 어플리케이션의 경우 이러한 왜곡을 먼저 수정해야합니다. 이러한 매개 변수를 찾으려면 잘 정의 된 패턴 (예 : 체스 보드)의 샘플 이미지를 제공해야합니다. 우리는 상대 위치 (예 : 체스 보드의 사각형 코너)를 이미 알고있는 몇 가지 구체적인 점을 찾습니다. 우리는 실세계 공간에서이 점들의 좌표를 알고 있고 이미지의 좌표를 알고 있기 때문에 왜곡 계수를 풀 수 있습니다. 더 나은 결과를 얻으려면 적어도 10 개의 테스트 패턴이 필요합니다.

#### Code
앞서 언급했듯이 카메라 교정을 위해 최소한 10 가지 테스트 패턴이 필요합니다. OpenCV에는 체스 보드 이미지 (samples / data / left01.jpg - left14.jpg 참조)가 제공되므로이를 활용할 것입니다. 체스 보드의 이미지를 생각해보십시오. 카메라 보정에 필요한 중요한 입력 데이터는 3D 실세계 점 세트와 이미지에서 이러한 점의 해당 2D 좌표입니다. 2D 이미지 포인트는 이미지에서 쉽게 찾을 수 있습니다. (이 이미지 포인트는 두 개의 검정색 사각형이 체스 보드에서 서로 닿는 위치 임)

실제 공간에서 3D 점은 어떻습니까? 이러한 이미지는 고정 카메라에서 가져오고 체스 보드는 다른 위치와 방향에 배치됩니다. 그래서 \\((X, Y, Z)\\) 값을 알아야합니다. 그러나 간단히 말하면, 체스 판이 XY 평면에 고정되어 있다고 말할 수 있습니다 (따라서 Z = 0 항상). 따라서 카메라는 그에 따라 이동되었습니다. 이 고려 사항은 X, Y 값만 찾을 수있게 해줍니다. 이제 X, Y 값에 대해 포인트를 (0,0), (1,0), (2,0), ...으로 전달하면 포인트의 위치를 ​​나타낼 수 있습니다. 이 경우 우리가 얻는 결과는 체스 보드 스퀘어 크기의 척도가됩니다. 그러나 사각형 크기 (30mm)를 알면 값을 (0,0), (30,0), (60,0), ...으로 전달할 수 있습니다. 따라서 우리는 결과를 mm로 얻습니다. (이 경우 이미지를 찍지 않았으므로 정사각형 크기를 알지 못하므로 정사각형 크기로 전달합니다.)

3D 점을 객체 점이라고하고 2D 이미지 점을 이미지 점이라고합니다.

In [ ]:
print(66)